# Chain

**Chain**(체인)은 여러 컴포넌트(요소)를 정해진 순서대로 연결하여 **복잡한 AI 작업을 단계별로 자동화**할 수 있도록 돕는 구조이다.

- 각 컴포넌트는 **이전 처리결과를 입력으로 받아 처리한 후 다음 단계로 결과를 전달**한다.
- 복잡한 작업을 여러 개의 단순한 단계로 나누고, 각 단계를 순차적으로 실행함으로써 전체 작업을 체계적으로 구성할 수 있다.

## 기본 개념

- 체인은 하나의 LLM 호출에 그치지 않고 **여러 LLM 호출이나 도구 실행등을 순차적으로 연결**하여 실행 할 수 있다.
- 예를 들어, 사용자의 질문 → 검색 → 요약 → 응답 생성 같은 일련의 작업을 체인으로 구성할 수 있다.
- 이러한 체인구조를 사용하면 **작업흐름이 명확**해지고 **코드의 재사용성**이 높아지며 **유지 보수 및 확장성이** 향상된다.

## LangChain에서의 Chain 구성 방식

LangChain은 다음 두 가지 방식을 통해 체인을 구성할 수 있다.

### 1. Off-the-shelf Chains 방식 (클래식 방식)

- LangChain에서 제공하는 **미리 정의된 Chain 클래스**(예: `LLMChain`, `SequentialChain`, `SimpleSequentialChain`)를 활용하는 방식이다.
- 각 클래스는 다양한 chain 알고리즘들을 미리 구현한 것으로 상황에 맞는 것을 선택하여 필요한 구성요소를 전달해 생생한다.
- 이 방식은 LangChain의 **초기 방식**이며, 새로운 기능 확장이나 유연한 구성에 한계가 있기 때문에 현재 **더 이상 사용되지 않음(deprecated)** 상태이다.
  - 현재 LangChain에서는 권장하지 않는 방식이다.

### 2. LCEL (LangChain Expression Language) 방식

- LCEL은 체인을 표현식(Expression) 기반의 선언적 파이프라인 방식으로 구성할 수 있도록 설계된 최신 체인 구성 방법이다. 
- 각 컴포넌트들을 `|` 연산자로 연결하여, 흐름이 자연스럽게 이어지는 형태의 체인을 구성한다.
- LCEL 방식은 간결하고 선언적인 문법을 제공하여 **직관적이고 융통성과 확장성 있는 체인 구성**이 가능하다.
- LCEL은
  - 선형적 흐름 구조를 가진다.
  - 문법이 간결하고 선언적이다.
  - 체인의 구조가 코드만 봐도 쉽게 파악된다.
  - 유연하고 확장성이 매우 뛰어나다.
- `Runnable` 기반 구조
  - LCEL방식을 구성하는 모든 컴포넌트들은 `Runnable` 이라는 공통 인터페이스를 기반으로 동작한다.
  - 체인을 구성하는 각 컴포넌트들은 `Runnable` 을 상속하여 구현하여 이를 통해 일관된 실행 인터페이스를 제공한다.
  - **공통 메소드**:
    - `invoke()`: 단일 입력에 대한 처리
    - `batch()`: 다수 입력을 묶어서 한번에 처리
    - `stream()`: 스트리밍 방식의 요청
    - `ainvoke()`, `abatch()`, `astream()`: 비동기적 처리 메소드

In [2]:
# 기존 off the shelf
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv()
prompt=ChatPromptTemplate.from_template(
    template="{item}에 어울리는 브랜드 이름 {count}개를 만들어 주세요."
)
model = ChatOpenAI(model="gpt-5-mini")
parser = StrOutputParser()
# input_variable->(프롬프트)->쿼리->(모델)->답변->(파서)->최종 답변


In [ ]:
# 기존 off the shelf
from langchain_classic import LLMChain
# Chain을 구성하는 요소들을 넣어서 생성.
chain = LLMChain( # prompt->model->parser 기본체인을 구성.
    prompt=prompt,
    llm=model,
    output_parser=parser
)
# 첫번째:Prompt -> Prompt에 전달할 값
res = chain.invoke({"item":"가방", "count":3}) 
print(res['text'])

다음 세 가지 제안드립니다. 각 이름에 간단한 콘셉트와 태그라인도 붙였습니다.

1) 벨루나 (Belluna)  
- 콘셉트: 모던하고 고급스러운 가죽·미니멀 라인 브랜드  
- 태그라인: 심플의 품격

2) 시티노마 (Citynoma)  
- 콘셉트: 도시 생활에 맞춘 실용적이고 다용도인 데일리 백  
- 태그라인: 도시를 누비는 동반자

3) 그린리프 (GreenLeaf / 그린리프)  
- 콘셉트: 재생 소재·친환경 제작을 강조한 에코 브랜드  
- 태그라인: 지구를 담는 가방

원하시면 타겟 연령대(예: 20대 초중반/직장인 등)나 브랜드 톤(럭셔리/캐주얼/빈티지 등)에 맞춰 이름을 더 제안드릴게요.


In [8]:
# LCEL
chain2 = prompt | model | parser # '|' 연산자 오버라이딩을 해둠.
print(type(chain2))
res2 = chain2.invoke({"item":"AI Agent 공부해야 할 것", "count":3})
print(res2)

<class 'langchain_core.runnables.base.RunnableSequence'>
아래처럼 3가지 제안드립니다. 각 이름은 발음·브랜딩·목적성을 고려해 짧게 설명을 붙였습니다.

1) 공부메이트 (GongbuMate)  
   - 친근하고 직관적인 이름으로, 학생용 AI 학습 도우미 이미지에 잘 어울립니다.

2) 에이듀 (AiDu / 에이듀)  
   - AI + Education 결합형 네이밍. 짧고 현대적이며 브랜드화하기 쉽습니다.

3) 루미AI (LumiAI / 루미에이아이)  
   - '빛(루미)'으로 깨달음·통찰을 상징하는 느낌의 프리미엄·개인화 학습 서비스에 적합합니다.

원하는 분위기(캐주얼/전문적/국문중심/영문중심)가 있으면 그에 맞춰 더 만들어 드릴게요.


# Runnable 타입 주요 클래스

## [Runnable](https://reference.langchain.com/python/langchain_core/runnables/#langchain_core.runnables.base.Runnable)
- LangChain의 Runnable은 실행 가능한 작업 단위를 캡슐화한 개념으로, 데이터 흐름의 각 단계를 정의하고 **체인(chain) 에 포함 되어**  복잡한 작업의 각 단계를 수행 한다.
- Chain을 구성하는 class들은 Runnable의 상속 받아 구현한다.
- **Prompt Template클래스**, **Chat 모델, LLM 모델 클래스**, **Output Parser 클래스** 등 다양한 컴포넌트가 Runnable을 상속받아 구현된다.

### 주요 특징
- 작업 단위의 캡슐화:
    - Runnable은 특정 작업(예: 프롬프트 생성, LLM 호출, 출력 파싱 등)을 수행하는 독립적인 컴포넌트이다.
    - 각 컴포넌트는 독립적으로 테스트 및 재사용이 가능하며, 조합하여 복잡한 체인을 구성할 수 있다.
- 체인 연결 및 작업 흐름 관리:
    - Runnable은 체인(chain, 일련의 연결된 작업 흐름)을 구성하는 기본 단위로 사용된다.
    - LangChain Expression Language(LCEL)를 사용하면 | 연산자를 통해 여러 Runnable을 쉽게 연결할 수 있다.
    - 입력과 출력의 형식을 일관되게 유지하여 각 단계가 자연스럽게 연결된다.
- 모듈화 및 디버깅 용이성:
    - 각 단계가 명확히 분리되어 문제 발생 시 어느 단계에서 오류가 발생했는지 쉽게 확인할 수 있다.
    - 복잡한 작업을 작은 단위로 나누어 체계적으로 관리할 수 있다.
      
### Runnable의 표준 메소드
- 모든 Runnable이 구현하는 공통 메소드
    - **`invoke(input, config:RunnableConfig)->output`**: 단일 입력을 처리하여 결과를 반환.
    - **`batch(input:list, config:RunnableConfig|list[RunnableConfig]) -> list[Output]`**: 여러 입력 데이터들을 한 번에 처리.
    - **`stream(input, config:RunnableConfig) -> Iterator[Output]`**: 입력에 대해 스트리밍 방식으로 응답을 반환.
    - **`assign(**kwargs)`**:
      -  앞 Runnable의 출력 결과에 새로운 key–value 쌍의 Field 추가(assign) 하여 다음 Runnable로 전달.
      -  값으로는 Runnable 객체(LCEL체인등)나 고정 값(리터럴) 모두 가능하며, 각 항목은 실행 시 평가되어 기존 출력에 병합한다.
      -  주로 앞 단계의 출력에 부가 정보(field)를 추가하고자 할 때 사용한다. 특히 `RunnablePassthrough`와 결합해, 입력을 그대로 넘기면서 특정 field만 추가할 때 자주 사용
### Runnable의 주요 구현체(하위 클래스)

- **`RunnableSequence`**
    - 여러 `Runnable`을 순차적으로 연결하여 실행하는 구성이다.
    - 각 단계의 출력이 다음 단계의 입력으로 전달된다.
    - 보통은 LCEL 문법을 사용해서 정의한다.
      - LCEL을 사용하여 체인을 구성할 경우 자동으로 `RunnableSequence`로 변환된다.
  
-  **`RunnablePassthrough`**
    - 입력 데이터를 가공하지 않고 그대로 다음 단계로 전달하는 `Runnable`이다.
      - 앞 Runnable으로 부터 전달 받은 **입력 값을 다음 Runnable로 그대로 전달**한다.
           - `RunnablePassthrough()`
      - 입력받은 값에 **Field를 추가**해서 전달할 경우 `assign()` 메소드를 사용한다.
           - `RunnablePassthrough.assign(new_key1="new_value1", new_key2="new_value2", ..)`

- **`RunnableParallel`**
    - 여러 `Runnable`을 병렬로 실행한 후, 결과를 결합하여 다음 단계로 전달한다.
    - 
        ```python
        RunnableParallel(
            {
                "key1":Runnable1, 
                "key2":Runnable2,
                "key3":Runnable3, ...
            }
        )
        ```
      - 각 Runnable의 실행결과를 Value로 Dictionary를 생성해서 반환한다.
      - LCEL로 정의할 때는 Chain에 dictionary로 정의한다.

- **`RunnableLambda`**
    - 일반 함수를 `Runnable`로 변환할 때 사용한다.
    
    - Runnable을 입력해야 하는 자리에 함수를 넣어야 하는 경우 RunnableLambda로 그 함수를 Runnable로 만들어 넣는다.
    - **구현**
      1. `RunnableLambda(함수객체)`
      2. `@chain` decorator를 이용해 함수를 `RunnableLamda`로 구현할 수있다.
      3. LCEL 에 함수를 포함시키면 `RunnableLambda`로 자동 변환된다.

#### Runnable 예제

In [17]:
from langchain_core.runnables import Runnable

class MyRunnable(Runnable):
    # invoke(), stream(), batch(), assign() 중에서 필요한 메소드를 재정의.
    def invoke(self, input_data, config=None):
        # input_data : 처리를 위한 입력값. 1개. 여러개면 리스트로 묶어야됨.
        # config: RunnableConfig -> 입력시에는 dict
        # Runnable이 실행할 때 필요한 설정정보를 입력.
        #   - {"Configurable":{key: value}}
        output = ""
        if config is not None:
            if config['configurable'].get("lang") == "en":
                output = "\n\n답변은 영어로 해주세요."
        
        return f"{input_data}에 대해서 한 문장으로 정의해주세요.{output}"

In [18]:
mr1 = MyRunnable()
print(mr1.invoke('사과'))
print(mr1.invoke("컴퓨터", {"configurable":{"lang":"en"}}))

사과에 대해서 한 문장으로 정의해주세요.
컴퓨터에 대해서 한 문장으로 정의해주세요.

답변은 영어로 해주세요.


In [20]:
# chain
chain = mr1 | model
res = chain.invoke("사과", {"configurable":{"lang":"en"}})
print(res.content)

An apple is the pomaceous fruit of the apple tree (Malus domestica), typically round, crisp, and sweet-to-tart in flavor, commonly eaten raw or used in cooking and baking.


#### RunnableLambda 예제

In [25]:
from langchain_core.runnables import RunnableLambda

mr2 = RunnableLambda(lambda input_data : f"{input_data}를 한문장으로 설명해줘.")
isinstance(mr2, Runnable)

True

In [ ]:
query = mr2.invoke("LLM")
res = (mr2|model).invoke("LLM")

AIMessage(content='LLM(대형 언어 모델)은 대규모 텍스트를 학습해 문맥에 맞는 단어(토큰)를 확률적으로 예측·생성하여 자연어 이해·생성, 번역, 요약 등 다양한 언어 작업을 수행하는 인공지능 모델입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 202, 'prompt_tokens': 17, 'total_tokens': 219, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CnyCquMjLLsHDij9lGqvxiC6RlqvL', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b2f53-2f92-7be1-a90a-0032f15384e8-0', usage_metadata={'input_tokens': 17, 'output_tokens': 202, 'total_tokens': 219, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 128}})

In [31]:
def sum(input_data):
    return input_data + 100
mr3 = RunnableLambda(sum)
query = mr3.invoke(9999)
print(query)

10099


In [ ]:
# Runnable로 만들 함수
# 파라미터: 1개
# 반환값 : 다음 체인구성에 전달할 값의 형식.

In [ ]:
prompt = ChatPromptTemplate.from_template(
    template="{subject}에 대해 100글자 이내로 설명해줘."
)
model = ChatOpenAI(model="gpt-5-mini")
parser = StrOutputParser()

# 출력결과, 글자수
chain = prompt | model | parser | RunnableLambda(lambda input_data: (input_data, len(input_data)))
res = chain.invoke({"subject":"AI Agent 구성요소"})

In [36]:
print(res)

('지각(센서), 상태·메모리, 목표·보상, 계획·정책, 실행·액터, 학습 모듈.', 43)


In [37]:
def get_length(value):
    return value, len(value)

chain2 = prompt | model | parser | get_length # get_length가 내부적으로 Runnable로 바뀜.
res = chain.invoke({"subject":"AI Agent 원리"})
print(res)

('환경을 인식해 모델·정책으로 계획·행동하고 보상으로 학습해 목표 달성.', 39)


#### RunnablePassThrough 예제

In [44]:
from langchain_core.runnables import RunnablePassthrough

rp = RunnablePassthrough()
res = rp.invoke("안녕하세요")
res = rp.invoke({"a":10, "b":20})
print(res)

{'a': 10, 'b': 20}


In [57]:
# 입력받은 값에 field를 추가해서 전달. 입력: dict, item을 추가해서 전달

address_runnable = RunnableLambda(lambda x: "서울시 금천구")
phone_runnable = RunnableLambda(lambda x: "010-1234-5678")

rp2 = RunnablePassthrough.assign(
    address=address_runnable, phone=phone_runnable
    #변수=Runnable(Callable) key:변수-Value:Runnable 반환값을 input dict에 추가.s
)
res = rp2.invoke({"name":"홍길동"})
res

{'name': '홍길동', 'address': '서울시 금천구', 'phone': '010-1234-5678'}

#### RunnableParallel 예제

In [60]:
from langchain_core.runnables import RunnableParallel

run1 = RunnableLambda(lambda x : x + 10)
run2 = RunnableLambda(lambda x : x * 10)
run3 = RunnableLambda(lambda x : x ** 10)

parallel = RunnableParallel(
    {
        "value1":run1, #run1의 실행결과를 "value1" key에 담아서 반환.
        "value2":run2,
        "value3":run3,
        "value4":RunnablePassthrough() # value4:입력값
    }
)
res = parallel.invoke(2)
res

{'value1': 12, 'value2': 20, 'value3': 1024, 'value4': 2}

In [72]:
from operator import itemgetter
# chain = RunnableLambda(lambda x : x['v1']) | parallel
chain = itemgetter('v1') | parallel
res = chain.invoke({"v1":10, "v2":20})
res

{'value1': 20, 'value2': 100, 'value3': 10000000000, 'value4': 10}

In [71]:
from operator import itemgetter

ig = itemgetter('v2') # 자료구조에서 "v1" 값을 조회
d = {"v1":10, "v2":20}
ig(d)

ig2 = itemgetter(2) # 자료구조[2] 조회한 결과를 반환.
ig2([1,2,3,4])
ig2((10,20,30,40))

30

In [73]:
# LCEL Chain안에서 RUnnableParellel 표현식. {key:Runnable}
chain = run1 | {'result1':run2, "result2":run3}
chain.invoke(20)

{'result1': 300, 'result2': 590490000000000}

In [ ]:
# {}와 Runnable이 '|'로 묶이면 {}이 Runnable
chain = {'result1':run2, "result2":run3} | RunnablePassthrough()
chain.invoke(2)

{'result1': 20, 'result2': 1024}

In [ ]:
rp = {'result1':run2, "result2":run3} # 이건 그냥 딕셔너리다. invoke가 안된다.
rp.invoke()

### LCEL Chain 예제

In [ ]:
# 입력 : 음식 이름
# 출력 : 음식의 레시피
# chain : prompt_template -> model(gpt-5-mini) -> StrOutputParser

In [2]:
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

prompt = ChatPromptTemplate.from_template(
    template=""" # Instrunction
당신은 숙련된 요리 전문 assistant 입니다.
요청한 음식의 레시피를 자세하게 작성해 주세요.

# Input Data
음식이름: {food}

# Output Indicator
- markdown 형식으로 작성한다.
- 다음 항목들을 넣어 레시피를 작성한다.
    - 요리 이름
    - 요리 기본 정보
        - 난이도
        - 조리시간
        - 인분
    - 재료
    - 요리 방법
    - 팁
"""
)
model = ChatOpenAI(model="gpt-5-mini")
parser = StrOutputParser()

recipe_chain = prompt | model | parser
res = recipe_chain.invoke({"food":"바스크치즈케이크"})

In [89]:
print(res)

# 요리 이름
순두부찌개

# 요리 기본 정보
- 난이도: 중간  
- 조리시간: 총 25–35분 (준비 10–15분 / 조리 15–20분)  
- 인분: 2–3인분

# 재료
(기본 조합 — 돼지고기 버전)
- 순두부 1봉(약 300–400g)  
- 돼지고기(다진 목살 또는 얇게 썬 삼겹/목살) 100g  
- 양파 1/2개 (작게 썰기)  
- 대파 1대 (송송 썰기)  
- 마늘 2–3쪽 (다진 것)  
- 청양고추 1개(선택, 어슷썰기)  
- 고춧가루 1–1.5 큰술  
- 고추장 1 작은술(선택, 더 깊은 맛 원할 때)  
- 간장 또는 국간장 1 큰술  
- 소금, 후추 약간  
- 식용유 1 큰술  
- 참기름 1 작은술  
- 계란 1개  
- 멸치·다시마 육수 600ml(약 3컵) — 없으면 물 사용 가능  

(해산물 버전 대체 재료)
- 조개류(바지락/조개) 120–150g, 새우 6–8마리 등  
- 돼지고기 대신 해산물 사용 시 멸치육수 필수

(채식 버전)
- 멸치·육수 대신 다시마+표고버섯 우린 육수 사용, 고기·해산물 생략

# 요리 방법
1. 육수 준비  
   - 멸치·다시마 육수: 냄비에 물 800ml에 마른 멸치(頭·내장 제거) 10마리, 다시마 한 장을 넣고 중불로 5–8분 끓인 뒤 다시마는 꺼내고 약한 불로 2–3분 더 끓여 체에 걸러 600ml 준비.  
   - 간편할 경우 물 600ml 사용 가능(맛은 다소 약함).

2. 재료 손질  
   - 돼지고기: 한입 크기로 썰거나 다진 것 준비.  
   - 양파, 마늘, 대파, 청양고추 손질. 순두부는 봉지째로 숟가락으로 떠낼 준비.

3. 베이스 향내 내기  
   - 뚝배기(또는 냄비)에 식용유 1큰술과 참기름 조금을 두르고 중불에서 돼지고기를 넣어 겉면이 익을 때까지 볶는다.  
   - 다진 마늘과 양파를 넣고 투명해질 때까지 볶는다.

4. 고춧가루 기름에 볶기(향 살리기)  
   - 불을 약간 줄이고 고춧가루 1–1.5큰술과 고추장 1작은술(사용 시)을

In [137]:
resp2 = ""
for token in chain.stream({"food":"바스크 치즈 케이크"}):
    print(token, end="")
    resp2 += token

# 바스크 치즈 케이크 (Burnt Basque Cheesecake)

## 요리 기본 정보
- 난이도: 중간 (베이킹 경험이 있으면 쉬움)
- 조리시간: 준비 15분 + 굽기 45–60분 + 식힘/냉장 4시간 이상 (하루 전 준비 권장)
- 인분: 8인분 (지름 20cm 스프링폼 기준)

## 재료
- 크림치즈(실온) 600g  
- 설탕 180g (약 3/4컵 + 2큰술)  
- 큰 달걀 3개 (실온)  
- 생크림(휘핑 크림) 240ml (1컵)  
- 박력분 또는 옥수수전분 15g (약 1큰술) — 질감 조절용  
- 바닐라 익스트랙 1작은술 (또는 바닐라 빈 1/2대 씨)  
- 소금 한 꼬집 (약 1/4작은술)

도구:
- 지름 20cm 스프링폼 또는 틀  
- 파치먼트(베이킹)페이퍼 — 틀 높이보다 크게 잘라 사용  
- 전기믹서(또는 핸드믹서) 및 고무주걱

(옵션) 장식: 신선한 베리, 소금 카라멜, 휘핑크림 등

## 요리 방법
1. 오븐 예열 및 틀 준비  
   - 오븐을 220°C(430°F)로 예열합니다. (일부 오븐은 200–220°C 권장)  
   - 스프링폼 틀 바닥과 옆면을 파치먼트로 감싸되, 페이퍼가 틀 윗부분보다 4–6cm 더 나오게 높게 세워 둡니다. 바스크 치즈케이크는 윗부분이 깊게 갈색이 나며 페이퍼가 꼭 필요합니다.

2. 크림치즈 준비  
   - 실온의 크림치즈를 큰 볼에 넣고 전기믹서로 부드럽게 풀어줍니다(뭉친 덩어리 없도록). 약 1–2분.

3. 설탕 추가 및 섞기  
   - 설탕을 넣고 크림치즈와 완전히 섞일 때까지 약 1–2분 더 휘핑합니다. 너무 오래 휘핑하면 공기가 많이 들어가므로 중간 세기로 필요 부분만 섞습니다.

4. 계란 넣기  
   - 계란을 한 번에 한 개씩 넣고, 각 계란이 거의 흡수될 때까지 저속으로 섞습니다. 과도한 공기 유입을 막기 위해 고속으로 오래 휘핑하지 마세요.

5. 생크림과 바닐라, 소금, 전분 넣기  
   - 생크림과 바닐라, 소금을 넣고 고무주걱이나 저속으로 부드

In [93]:
print(resp2)

# 크림 치즈 파스타

## 요리 기본 정보
- 난이도: 쉬움
- 조리시간: 약 25–30분
- 인분: 2–3인분

## 재료
- 파스타(스파게티 또는 페투치네 추천): 200g  
- 크림치즈: 100–150g (실온에서 약간 부드럽게 한 것)  
- 생크림(선택): 100ml — 더욱 부드러운 소스를 원할 때  
- 버터: 1큰술(약 15g)  
- 올리브유: 1큰술  
- 마늘: 2쪽(다진 것)  
- 양파: 1/2개(얇게 다진 것, 선택)  
- 파르미지아노·레지아노(파마산) 치즈: 30–50g(갈아놓은 것)  
- 소금: 파스타 삶는 물에 넉넉히(끓는 물 1L당 소금 약 10g 권장) + 간 맞출 때 약간  
- 흑후추: 약간(갓 갈아 사용 권장)  
- 넛맥(옵션): 아주 약간(한 꼬집)  
- 파스타 삶은 물: 약 1컵(약 200–250ml) — 소스 농도 조절용으로 꼭 남겨둔다  
- 파슬리나 쪽파(장식용): 약간  
- 레몬 제스트 또는 레몬즙(옵션, 산미 추가): 약간  
- 추가 옵션 재료 (원하면)
  - 베이컨/판체타: 50–80g(잘게 썰어 바삭하게)  
  - 버섯: 100g(슬라이스)  
  - 시금치 한 줌

## 요리 방법
1. 준비  
   - 크림치즈는 미리 실온에 꺼내 두어 부드럽게 만든다(녹이기 쉬움).  
   - 파르미지아노는 갈아둔다.  
   - 마늘·양파 등 필요한 재료는 다진다.

2. 파스타 삶기  
   - 큰 냄비에 물을 넣고 강하게 끓인다. 물이 팔팔 끓으면 소금을 넉넉히 넣고 파스타를 넣어 포장지에 적힌 시간보다 1분 정도 덜 삶아 알덴테(살짝 덜 익음)로 건진다.  
   - 파스타를 건질 때 병아리 훌쩍 떠오르는 정도로 약 1컵(200–250ml) 정도의 삶은 물을 따로 덜어둔다(소스 농도 조절용).

3. 소스 베이스 만들기  
   - 팬(중간 크기)에 올리브유 1큰술과 버터 1큰술을 넣고 중약불로 가열한다.  
   - (베이컨/판체타 사용 시) 베이컨을 넣고 바삭하게 구워 꺼내 놓고, 남은 

In [3]:
# TODO
## 입력: 번역할 내용, 언어
## 출력: "번역할 내용"을 해당 "언어"로 번역한 결과
# 
# 입력 : {"content":"안녕하세요", "language":"영어"}
# 출력 : Hi
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

prompt_trans = ChatPromptTemplate.from_template(
    template=""" # Instrunction
당신은 외계어와 다국어가 가능한 순련된 번역 전문 assistant 입니다.
번역할 내용을 번역할 언어로 번역해 주세요.

# Input Data
번역할 내용: {content}
번역할 언어: {language}
"""
)
model = ChatOpenAI(model="gpt-5-mini")
parser = StrOutputParser()

translate_chain = prompt_trans | model | parser
translate_res = translate_chain.invoke({"content":"감사합니다", "language":"화성어"})

In [ ]:
print(translate_res)

Thra'kesh


In [133]:
resp2

'# 크림 치즈 파스타\n\n## 요리 기본 정보\n- 난이도: 쉬움\n- 조리시간: 약 25–30분\n- 인분: 2–3인분\n\n## 재료\n- 파스타(스파게티 또는 페투치네 추천): 200g  \n- 크림치즈: 100–150g (실온에서 약간 부드럽게 한 것)  \n- 생크림(선택): 100ml — 더욱 부드러운 소스를 원할 때  \n- 버터: 1큰술(약 15g)  \n- 올리브유: 1큰술  \n- 마늘: 2쪽(다진 것)  \n- 양파: 1/2개(얇게 다진 것, 선택)  \n- 파르미지아노·레지아노(파마산) 치즈: 30–50g(갈아놓은 것)  \n- 소금: 파스타 삶는 물에 넉넉히(끓는 물 1L당 소금 약 10g 권장) + 간 맞출 때 약간  \n- 흑후추: 약간(갓 갈아 사용 권장)  \n- 넛맥(옵션): 아주 약간(한 꼬집)  \n- 파스타 삶은 물: 약 1컵(약 200–250ml) — 소스 농도 조절용으로 꼭 남겨둔다  \n- 파슬리나 쪽파(장식용): 약간  \n- 레몬 제스트 또는 레몬즙(옵션, 산미 추가): 약간  \n- 추가 옵션 재료 (원하면)\n  - 베이컨/판체타: 50–80g(잘게 썰어 바삭하게)  \n  - 버섯: 100g(슬라이스)  \n  - 시금치 한 줌\n\n## 요리 방법\n1. 준비  \n   - 크림치즈는 미리 실온에 꺼내 두어 부드럽게 만든다(녹이기 쉬움).  \n   - 파르미지아노는 갈아둔다.  \n   - 마늘·양파 등 필요한 재료는 다진다.\n\n2. 파스타 삶기  \n   - 큰 냄비에 물을 넣고 강하게 끓인다. 물이 팔팔 끓으면 소금을 넉넉히 넣고 파스타를 넣어 포장지에 적힌 시간보다 1분 정도 덜 삶아 알덴테(살짝 덜 익음)로 건진다.  \n   - 파스타를 건질 때 병아리 훌쩍 떠오르는 정도로 약 1컵(200–250ml) 정도의 삶은 물을 따로 덜어둔다(소스 농도 조절용).\n\n3. 소스 베이스 만들기  \n   - 팬(중간 크기)에 올리브유 1큰술과 버터 1큰술을 넣고 중약불로 가열한다.  \n   - (

In [134]:
resp3 = translate_chain.invoke({"content":resp2, "language":"영어"})

In [135]:
print(resp3)

Cream Cheese Pasta

Basic information
- Difficulty: Easy
- Cooking time: About 25–30 minutes
- Servings: 2–3

Ingredients
- Pasta (spaghetti or fettuccine recommended): 200 g
- Cream cheese: 100–150 g (softened slightly at room temperature)
- Heavy cream (optional): 100 ml — for a creamier sauce
- Butter: 1 tbsp (about 15 g)
- Olive oil: 1 tbsp
- Garlic: 2 cloves (minced)
- Onion: 1/2 (thinly chopped, optional)
- Parmigiano-Reggiano (Parmesan) cheese: 30–50 g (grated)
- Salt: plenty for the pasta cooking water (about 10 g per 1 L of boiling water recommended) + a little for seasoning
- Black pepper: to taste (freshly ground recommended)
- Nutmeg (optional): a very small pinch
- Pasta cooking water: about 1 cup (200–250 ml) — reserve for adjusting sauce consistency
- Parsley or scallions (for garnish): a little
- Lemon zest or lemon juice (optional, for brightness): a little
- Optional add-ins (if desired)
  - Bacon/pancetta: 50–80 g (cut into small pieces and cooked until crispy)
  - M

### Chain과 Chain간의 연결

In [ ]:
type(chain)

In [ ]:
# recipe_chain - translate_chain(연결)
# 음식 레시피를 원하는 언어로 출력.
# recipe_chain : 입력 - food: 음식이름
# translate_chain : 입력 - content: 번역할 내용, language: 언어

In [ ]:
from operator import itemgetter
# chain입력: {food:음식이름, language: 언어}
# 첫번째 chain:food
# language는 두번째 chain으로 넘겨야 한다.
# 첫번째 chain의 출력결과를 content로 두번째 chain으로 넘긴다.
chain = {
    "content":recipe_chain,
    "language":itemgetter("language")
} | translate_chain

result = chain.invoke({"food":"바스크치즈케이크", "language":"영어"})

# 첫번째 문제. 체인들이 입력이 다름.
# c.invoke({"food":"된장찌개", "language":"영어", "content":"번역할 내용"})
# 두번째 문제
# 인자들이 첫번째 체인으로만 다 들어감. 두번쨰 체인으로는 안들어감.

In [8]:
# print(result)
from IPython.display import Markdown
Markdown(result)

# Basque Burnt Cheesecake

Basic info
- Difficulty: Medium  
- Time: Prep 15–25 minutes / Bake 45–55 minutes / Chill at least 4 hours (recommended: overnight) — total about 5 hours or more  
- Serves: 6–8 (20 cm / 8-inch cake pan)

Ingredients
- Cream cheese (full-fat, block) 500 g, at room temperature  
- Sugar 160 g  
- Large eggs 4, at room temperature  
- Heavy cream (35% fat or higher) 300 ml, at room temperature  
- Cake flour (or all-purpose flour) 20 g (about 2 tablespoons) — alternative: replace with 10 g cornstarch  
- Vanilla extract 1 teaspoon (or a little vanilla bean)  
- A pinch of salt (about 1/4–1/2 teaspoon)  
- (Optional) A little lemon zest — if you want brightness

Additional equipment: about a 20 cm (8-inch) springform or cake pan, baking paper (line it high)

Oven temperature reference: 210°C (about 410°F). If needed, use 200–220°C (390–430°F) depending on your oven.

Method
1. Preheat oven and prepare pan  
   - Preheat the oven to 210°C (adjust between 200–220°C / 390–430°F depending on your oven).  
   - Line the inside of the pan with baking paper, letting it rise well above the rim (paper should extend 5–8 cm / 2–3 in higher than the pan). You don’t need to fold or tape the paper; it will flatten during baking.

2. Bring ingredients to room temperature  
   - Take the cream cheese, eggs, and heavy cream out in advance so they are not cold (this helps avoid lumps).

3. Mix cream cheese and sugar  
   - Put the room-temperature cream cheese in a large bowl and soften it with a rubber spatula or hand mixer. Add the sugar and mix until smooth (use medium speed if using a mixer).

4. Add eggs one at a time  
   - Add the eggs one at a time, mixing completely after each addition. Adding them all at once can cause the mixture to split.

5. Add dry ingredients and flavoring  
   - Sift the cake flour (or cornstarch) and salt into the batter and fold gently with a spatula. Add vanilla and lemon zest (if using).

6. Add heavy cream and finish mixing  
   - Add the heavy cream in several additions, mixing until the texture is uniform. If using a mixer, work at low–medium speed and avoid overbeating to minimize air incorporation. The batter should be smooth and slightly loose (a viscous, creamy consistency).

7. Pour into pan and bake  
   - Pour the batter into the prepared pan and level the surface.  
   - Bake on the center rack of the preheated oven for 45–55 minutes. The surface should be a deep golden to very dark brown (sometimes nearly black) and well caramelized. The center should still jiggle slightly when shaken (a jelly-like wobble).

8. Cool and chill  
   - After baking, either crack the oven door open or remove the cake and let it cool completely in the pan at room temperature (about 1 hour). Then refrigerate for at least 4 hours, preferably overnight, so the center sets and the flavor improves.

9. Serve  
   - To remove from the pan, carefully lift the baking paper. The top will be deeply browned while the inside remains moist and creamy. Serve chilled or slightly warmed at room temperature for 20–30 minutes. Pairs well with fresh fruit, berry sauce, or caramel.

Tips
- Line the pan with baking paper higher than the rim: this prevents overflow when the batter rises and creates the characteristic deeply colored edge.  
- Bring all ingredients to room temperature: cold cream cheese makes lumps and prevents even mixing.  
- Do not overmix: incorporating too much air can make the cake rise a lot in the oven and then sink and crack when cooling.  
- Baking temperature and time: the signature “burnt” top comes from the high temperature, so bake between 200–220°C (390–430°F). If the surface browns too quickly, lower the temperature by 10–20°C and increase the bake time.  
- A slightly wobbly center at the end of baking is normal. It will set during chilling.  
- If it’s too dense: you may have overbaked or the batter was too thick. Next time shorten the bake time or lower the oven temperature.  
- Ingredient swaps: low-fat ingredients are not recommended. The rich fat content is what gives Basque cheesecake its texture.  
- Storage: Refrigerate 4–5 days. Freeze up to 1 month (thaw in the fridge before eating).

Enjoy making it! If you want, I can help scale the recipe to half size (for a smaller pan) or adjust quantities for a larger pan.

## 사용자 함수를 Chain에 적용하기

### 사용자 함수를 Runnable로 정의 (RunnableLambda)
- 임의의 함수를 Runnable로 정의 할 수있다.
  - chain에 포함할 기능을 함수로 정의할 때 주로 사용. 
- `RunnableLambda(함수)` 사용
  - 함수는 invoke() 메소드를 통해 입력받은 값을 받을 **한개의 파라미터**를 선언해야 한다.
  - 보통 Lambda 표현식으로 정의한 함수를 LCEL chain에 포함시킬 때 사용한다.
  
#### Runnable 에 사용할 **사용자 정의 함수** 구문
- 이전 Chain의 출력을 입력 받는 **파라미터를 한개** 선언한다. (첫번째 파라미터)
- `invoke()`로 호출 할때 전달 하는 추가 설정을 입력받는 파라미터를 선언한다.(두번째 파라미터 - Optional)
  - RunnableConfig 타입의 값을 받는데 Dictionary 형식으로 `{"configuable": {"설정이름":"설정값"}}` 형식으로 받는다.
- 만약 함수가 여러개의 인자를 받는 경우 단일 입력을 받아들이고 이를 여러 인수로 풀어내는 래퍼 함수를 작성하여 Runnable로 만든다.
  ```python
  def plus(num1, num2):
      ...

  def wrapper_plus(nums:dict|list):
      return plus(nums['num1'], nums['num2'])
  ```

### 함수 자체를 chain에 추가
- RunnableLambda에 전달할 함수 구문에 맞는 함수라면 RunnableLambda를 사용하지 않고 chain에 넣을 수 있다. 
- 단 함수로 정의하고 LCEL에 포함시켜야 한다. Lambda 표현식으로 작성한 함수는 `RunnableLambda`를 사용해야 한다.

### 사용자 함수를 Chain으로 정의
- Chain 을 구성하는 작업 사이에 추가 작업이 필요할 경우, 중간 결과를 모두 사용해야 하는 경우 함수로 구현한다.
- `@chain` 데코레이터를 사용해 함수에 선언한다.

In [15]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

def plus(num1, num2):
    return num1 + num2

def wrapper_plus(num:dict|list)->int:
    return plus(num[0], num[1])

# chain = RunnablePassthrough() | wrapper_plus
chain = RunnablePassthrough() | RunnableLambda(lambda num:plus(num[0], num[1]))
chain.invoke([10, 20])

30

In [4]:
from langchain_core.runnables import chain
# @chain:복잡한 체인구조를 정의한 함수를 Runnable로 만들때 사용.
# LCEL: 순차구조(흐름)만 정의가 가능.
# @chain 함수를 이용해서 chain 구성:조건문, 반복문을 이용해 chain의 흐름을 제어할 경우

@chain
def custom_chain(input_data:dict)->dict[str, str]:
    # input_data: food, language, kor_recipe:bool
    # kor_recipe: True 면 한국어 레시피와 language로 번역된 레시피 반환
    #           : False면 language로 번역된 레시피만 반환
    # 조건절은 LCEL 로는 구현이 안됨.
    food = input_data['food']
    language = input_data['language']
    is_kor = input_data['kor_recipe']

    korean_recipe = recipe_chain.invoke({"food":food})
    result = translate_chain.invoke(
        {"content":korean_recipe, "language":language}
    )
    if is_kor:
        return (korean_recipe, result)
    else:
        return result
res = custom_chain.invoke(
    {"food":"어남선생 리조또", "language":"영어", "kor_recipe":True})

In [5]:
print(res[0])

# 어남선생 리조또

해산물 베이스의 크리미한 리조또. 바다의 풍미를 살린 깔끔한 국물과 버터·파르메산의 조합으로 깊고 부드러운 식감을 냅니다.

## 요리 기본 정보
- 난이도: 중간
- 조리시간: 약 35–45분 (준비 10분, 조리 25–35분)
- 인분: 2–3인분

## 재료
- 쌀
  - 아르보리오(리조또용) 쌀 200g (약 1컵)
- 육수/국물
  - 생선·해물 육수(또는 치킨·야채 육수) 850–900ml(따뜻하게 유지)
- 해산물
  - 혼합 해산물(새우, 오징어, 홍합 등) 250–300g
- 채소/향미
  - 양파(중) 1개 → 다진 것
  - 마늘 2쪽 → 다진 것
  - 올리브유 2큰술
- 풍미/마무리
  - 드라이 화이트 와인 100ml (없으면 육수 100ml + 식초 약간으로 대체)
  - 버터 30g (마무리용 20g + 볶을 때 10g)
  - 파르메산 치즈 간 것 40–60g (간 보기)
  - 소금, 갓 간 흑후추 적당량
  - 다진 파슬리 한 줌
  - 레몬 1/2개 (즙 또는 제스트)
- 선택 재료 (한국적 변주)
  - 다시마·멸치로 만든 국물 900ml (해물 육수 대체 가능)
  - 고추기름 또는 고춧가루 약간(매콤한 변주)

## 요리 방법
1. 준비
   - 해산물은 깨끗이 씻어 물기를 빼고, 새우는 껍질을 벗긴 뒤 등 쪽 내장을 제거한다(머리/껍질은 육수 내면 풍미 좋음).
   - 육수는 냄비에 데워 약한 불에서 따뜻하게 유지한다(끓이지 않음).
   - 파르메산은 곱게 갈아 준비한다.

2. 해산물 전처리 (선택 사항)
   - 해산물을 팬에 올리브유 한 작은술과 마늘 약간으로 살짝 볶아 반쯤 익힌 뒤 따로 덜어둔다. (리조또에서 해산물을 마지막에 넣어 과도한 조리를 방지)

3. 소테(향내기)
   - 넓은 팬(또는 냄비)에 올리브유 2큰술과 버터 10g을 넣고 중약불로 가열한다.
   - 다진 양파를 넣고 투명해질 때까지 4–5분 볶는다. 다진 마늘을 넣고 30초 더 볶아 향을 낸다.

4. 쌀 넣기(

In [6]:
print(res[1])

Title: Mr. Eonam’s Seafood Risotto

A creamy risotto with a seafood base. Clean broth that highlights the flavors of the sea combined with butter and Parmesan gives a deep, smooth texture.

Basic information
- Difficulty: Medium
- Total time: about 35–45 minutes (10 minutes prep, 25–35 minutes cooking)
- Serves: 2–3

Ingredients
- Rice
  - Arborio (risotto) rice 200 g (about 1 cup)
- Stock/Broth
  - Fish/seafood stock (or chicken/vegetable stock) 850–900 ml, kept warm
- Seafood
  - Mixed seafood (shrimp, squid, mussels, etc.) 250–300 g
- Vegetables/aromatics
  - 1 medium onion → chopped
  - 2 garlic cloves → minced
  - Olive oil 2 tbsp
- Flavor/finishing
  - Dry white wine 100 ml (if unavailable, substitute 100 ml stock + about 1 tsp vinegar)
  - Butter 30 g (10 g for sautéing + 20 g for finishing)
  - Grated Parmesan 40–60 g (to taste)
  - Salt and freshly ground black pepper to taste
  - A handful of chopped parsley
  - 1/2 lemon (juice or zest)
- Optional (Korean variation)
  - Kelp

In [7]:
res = custom_chain.invoke(
    {"food":"어남선생 리조또", "language":"영어", "kor_recipe":False})

In [9]:
print(res)

Title: Eonam’s Risotto (Seafood Risotto)

Basic information
- Difficulty: Intermediate (requires attention to risotto texture and cooking seafood)
- Total time: about 40 minutes (10 min prep + 30 min cooking)
- Servings: 2–3

Ingredients
- Arborio rice (or Carnaroli) 250 g (about 1 cup + 1/4 cup)
- Mixed seafood 300 g (e.g., shrimp 120 g, squid 100 g, mussels/clams 80 g — adjust to taste)
- Fish/seafood stock 1.2 L (keep hot)
  - Alternative: chicken stock (if you want more seafood flavor, make stock with shrimp shells or anchovies + kelp)
- White wine 80–100 ml (if unavailable, substitute 100 ml chicken stock or water)
- Onion (medium) 1, or 2 shallots (finely chopped)
- Garlic 2 cloves (minced)
- Olive oil 2 tbsp
- Unsalted butter 30 g
- Parmigiano-Reggiano (or Parmesan), grated 50 g
- Lemon zest (optional) about 1 tsp + lemon juice about 1 tsp
- Salt and pepper to taste
- Chopped parsley (optional) 1 tbsp
- (Optional) Anchovy extract or fish sauce 1/2–1 tsp — for extra umami

Method

# Cache

- 응답 결과를 저장해서 같은 질문이 들어오면 LLM에 요청하지 않고 저장된 결과를 보여주도록 한다.
    - 처리속도와 비용을 절감할 수 있다.
    - 특히 chatbot같이 비슷한 질문을 하는 경우 유용하다.
- 저장 방식은 `메모리`, `sqlite` 등 다양한 방식을 지원한다.
  
    ```python
    set_llm_cache(Cache객체)
    ```

In [26]:
from langchain_community.cache import InMemoryCache, SQLiteCache
from langchain_core.globals import set_llm_cache

# set_llm_cache(InMemoryCache()) # 대화내역을 메모리에 저장. 껐다 키면 해제됨. 한번만 선언하면됨.
set_llm_cache(SQLiteCache(database_path="cache.sqlite")) # 대화내역을 SQLite DB에 저장. 디비파일 경로를 지정.

prompt = ChatPromptTemplate(
    [
        ("system", "당신은 유능한 assistant입니다. 공부를 원하는 과목에 대한 학습 커리큘럼을 자세히 짜주세요."),
        ("user", "{query}")
    ]
)
model = ChatOpenAI(model="gpt-5-mini")
chain = prompt | model | StrOutputParser()

In [27]:
chain.invoke({"query":"AI Agent에 대해 무엇을 공부해야 할지 알려줘."})

'아래는 “AI Agent(지능형 에이전트)”를 체계적으로 공부하기 위한 상세 커리큘럼입니다. 초심자 → 실무자 → 연구자 단계로 구분하고, 각 단계별 핵심 주제, 실습 과제, 추천 자료(강의·책·논문·라이브러리), 목표 및 평가 방법까지 포함했습니다. 본인을 위해 3개월·6개월·1년 로드맵도 제안합니다. 필요하면 학습 기간·관심 분야(로보틱스·대화형 LLM 에이전트·멀티에이전트 등)에 맞춰 커리큘럼을 맞춰드릴게요.\n\n요약(한눈에)\n- 핵심 영역: 확률·선형대수·최적화, 기초 머신러닝, 강화학습(RL), 플래닝·검색, 에이전트 아키텍처(반응형·계획형·하이브리드), LLM 기반 에이전트(도구 사용, 메모리, 체인 오브 소트), 다중 에이전트 시스템(MAS), 평가·안전성·배포\n- 실습 환경: Python, OpenAI Gym/MiniGrid, PettingZoo, Unity ML-Agents, Carla(자율주행), Hugging Face, LangChain, Stable-Baselines3, RLlib\n- 추천 프로젝트: Grid world↑자율 주행 시뮬레이션/대화형 에이전트, 도구 호출 LLM 에이전트, 멀티에이전트 협력/경쟁 실험\n\n1) 시작 전(필수 전제 조건)\n- 수학: 선형대수(행렬·고유값), 확률·통계(조건부확률·MAP·MLE), 미적분(편미분)\n- 프로그래밍: Python (numpy, pandas), 버전관리(Git)\n- 기초 ML: 지도학습(회귀·분류), 신경망 기본(퍼셉트론·역전파), PyTorch 또는 TensorFlow 기본\n\n2) 학습 로드맵(단계별 상세)\n\nA. 기초(2–4주)\n목표: 에이전트의 개념·환경·보상·행동 사이클 이해\n- 개념: 에이전트/환경/상태·행동·보상·정책·가치함수·에피소드\n- 탐색 주제: MDP(마르코프 결정 과정), 벨만 방정식\n- 실습: OpenAI Gym 기본 예제(CartPole, MountainCar)\n추천 자료:\n- 책: Sutton & Barto, “Reinforce

In [28]:
print(chain.invoke({"query":"AI Agent에 대해 무엇을 공부해야 할지 알려줘."}))

아래는 “AI Agent(지능형 에이전트)”를 체계적으로 공부하기 위한 상세 커리큘럼입니다. 초심자 → 실무자 → 연구자 단계로 구분하고, 각 단계별 핵심 주제, 실습 과제, 추천 자료(강의·책·논문·라이브러리), 목표 및 평가 방법까지 포함했습니다. 본인을 위해 3개월·6개월·1년 로드맵도 제안합니다. 필요하면 학습 기간·관심 분야(로보틱스·대화형 LLM 에이전트·멀티에이전트 등)에 맞춰 커리큘럼을 맞춰드릴게요.

요약(한눈에)
- 핵심 영역: 확률·선형대수·최적화, 기초 머신러닝, 강화학습(RL), 플래닝·검색, 에이전트 아키텍처(반응형·계획형·하이브리드), LLM 기반 에이전트(도구 사용, 메모리, 체인 오브 소트), 다중 에이전트 시스템(MAS), 평가·안전성·배포
- 실습 환경: Python, OpenAI Gym/MiniGrid, PettingZoo, Unity ML-Agents, Carla(자율주행), Hugging Face, LangChain, Stable-Baselines3, RLlib
- 추천 프로젝트: Grid world↑자율 주행 시뮬레이션/대화형 에이전트, 도구 호출 LLM 에이전트, 멀티에이전트 협력/경쟁 실험

1) 시작 전(필수 전제 조건)
- 수학: 선형대수(행렬·고유값), 확률·통계(조건부확률·MAP·MLE), 미적분(편미분)
- 프로그래밍: Python (numpy, pandas), 버전관리(Git)
- 기초 ML: 지도학습(회귀·분류), 신경망 기본(퍼셉트론·역전파), PyTorch 또는 TensorFlow 기본

2) 학습 로드맵(단계별 상세)

A. 기초(2–4주)
목표: 에이전트의 개념·환경·보상·행동 사이클 이해
- 개념: 에이전트/환경/상태·행동·보상·정책·가치함수·에피소드
- 탐색 주제: MDP(마르코프 결정 과정), 벨만 방정식
- 실습: OpenAI Gym 기본 예제(CartPole, MountainCar)
추천 자료:
- 책: Sutton & Barto, “Reinforcement Learning: An Int